# Cython

##  Cython is a **superset** of Python

* Cython is a **superset** of Python, with additional functionality   for defining C types and calling C functions
* Cython generates C wrapper code, which is compiled into a Python   extension module
* Major advantage: enables incremental code optimization
* Extensive documentation available on http://docs.cython.org

## `cdef`  is used to declare C variables

```cython
cdef int i, j, k
cdef float f, g[42], *h
```

## Cython function definitions

There are three kinds of Cython function definitions: `def`, `cdef` and `cpdef`:

```cython
# Python function.
def foo(int i, char *s):
    
# C function. Not visible to Python code that imports the module 
cdef int eggs(int i, float f):  

# "Hybrid". Generates both Python and C functions.
cpdef double foo_2(int i, float f):

```

**Note**: Function arguments and return types may be declared. 

## Cython optimises based on type definitions  

* If no type is specified for a variable, parameter or return type, it defaults to a Python object
* The standard Python for-loop is used in Cython:

```cython
cdef type int i, n

for i in range(n):
   ...
```   

* If `i` is declared as an integer (with `cdef int i`), this will be optimized into a standard C loop.

## A Cython example

* Approximate the integral of a general function `f(x)`
   <center>
    

![Integral of $f(x) = sin(x^2)$](figs/num_itg.png)

</center>


* Numerical integration: accuracy increases with number of intervals

* Speed is not a problem in 1D, but may be critical in 3D

## Cython example: Standard Python

Python implementation (not optimized) of the integration:

In [1]:
from math import sin


def f(x):
    return sin(x**2)


def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx


%timeit integrate_f(0, 2, 1_000_000)

913 ms ± 9.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Integration takes around 1 second with `N=1_000_000`.

## Cython example: Compilation with setuptools (recommended)

Compiling with setuptools is more convenient.

Make a script named `setup.py`:

```python
import numpy
from setuptools import setup
from Cython.Build import cythonize

setup(
    name='in3110-cython',
    ext_modules=cythonize("*.pyx"),
    include_dirs=[numpy.get_include()],
)
```

and compile the module with

In [2]:
!python3 setup.py build_ext --inplace

running build_ext
copying build/lib.macosx-11.0-arm64-cpython-310/apply.cpython-310-darwin.so -> 
copying build/lib.macosx-11.0-arm64-cpython-310/integral.cpython-310-darwin.so -> 
copying build/lib.macosx-11.0-arm64-cpython-310/integral_notypes.cpython-310-darwin.so -> 
copying build/lib.macosx-11.0-arm64-cpython-310/integral_types.cpython-310-darwin.so -> 


We can now import and run our compiled `integral` module

In [3]:
%pycat integral_notypes.pyx

from math import sin

def f(x):
    return sin(x**2)

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx


In [4]:
import integral_notypes

%timeit integral_notypes.integrate_f(0, 2, 1_000_000)

967 ms ± 77.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Cython example: Cython is only slightly faster than pure Python

<table border="1">
<thead>
<tr><th align="center">       Implementation        </th> <th align="center">Timing (normalised) </th> </tr>
</thead>
<tbody>
<tr> <td align="center">       Pure Python        </td> <td align="center">1.0 </td> </tr>
<tr> <td align="center">   Cython, no types              </td> <td align="center">   0.74    </td> </tr>
</tbody>
</table>

## Cython example: adding types

* Simply compiling the Cython file gives only minor speedup: loop runs in C, but makes numerous calls to the Python/C API
* To have any real speedup, we need to introduce types:

In [5]:
%pycat integral_types.pyx

from math import sin

cpdef double f(double x):
    return sin(x**2)

cpdef double integrate_f(double a, double b, int N):
    cdef double s = 0
    cdef double dx = (b - a) / N
    cdef int i
    for i in range(N):
        s += f(a + i * dx)
    return s * dx


In [6]:
import integral_types

%timeit integral_types.integrate_f(0, 2, 1_000_000)

183 ms ± 2.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Cython example: final version

* A fully typed version runs about 10 times faster:

```cython
from libc.math cimport sin  # Use cimport to make functions available to the C layer of Cython

cdef double f(double x):
    return sin(x**2)
```

In [7]:
import integral

%timeit integral.integrate_f(0, 2, 1_000_000)

2.8 ms ± 24.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Cython example: "less Python" equals "more speedup":

<table border="1">
<thead>
<tr><th align="center">       Implementation        </th> <th align="center">Timing (normalised) </th> </tr>
</thead>
<tbody>
<tr> <td align="center">       Pure Python        </td> <td align="center">1.0 </td> </tr>
<tr> <td align="center">   Cython, no types              </td> <td align="center">   0.74    </td> </tr>
<tr> <td align="center">   *double* + *int*    </td> <td align="center">   0.18    </td> </tr>
<tr> <td align="center">   Types and *math.h*       </td> <td align="center">   0.02    </td> </tr>
</tbody>
</table>

Speedup can be much higher, but requires slightly more complex example (loops within loops...).

You can also include your own C-functions, see http://cython.readthedocs.io/en/latest/src/tutorial/external.html.

# Cython and numpy

Cython works with numpy arrays as well.

### Example: Apply `sin` to all numbers in an array:

In [8]:
from math import sin

import numpy


def apply_sin(a):
    out = numpy.ndarray(len(a), dtype=numpy.double)

    for i in range(len(a)):
        out[i] = sin(a[i])

    return out

Usage:

In [9]:
a = numpy.linspace(0, 10, 1_000_000, dtype=numpy.double)
%timeit apply_sin(a)

723 ms ± 11.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Declaring numpy data types

Cython defines special data dtypes for numpy arrays. Below is the translation table between Python and Cython dypes:

| Numpy datatype| Cython datatype|
| ------------- |:-------------:|
| numpy.int8      | numpy.int8_t |
| numpy.int16      | numpy.int16_t |
| numpy.single      | numpy.single_t |
| numpy.double      | numpy.double_t |
| numpy.complex      | numpy.complex_t |


Defining a new numpy array in Cython:

```cython
cdef numpy.ndarray[numpy.double_t, ndim=1] out

out = numpy.zeros(1000, dtype=numpy.double)
```

# Declaring numpy data types

Below is a fully typed version of the `apply_sin` function:

In [10]:
%pycat apply.pyx

import numpy
cimport numpy

from libc.math cimport sin

cpdef numpy.ndarray[numpy.double_t, ndim=1] apply_sin(numpy.ndarray[numpy.double_t, ndim=1] a):
    cdef int i

    cdef numpy.ndarray[numpy.double_t, ndim=1] out
    out = numpy.ndarray(len(a), dtype=numpy.double)

    for i in range(len(a)):
        out[i] = sin(a[i])

    return out


## Using the Cython-numpy module

Save this file as `apply.pyx`. Once compiled, the cython module can be used as:

In [11]:
import apply

%timeit out = apply.apply_sin(a)

3.75 ms ± 40.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%timeit np.sin(a)

3.63 ms ± 45.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Timings

<table border="1">
<thead>
<tr><th align="center">       Implementation        </th> <th align="center">Timing (normalised) </th> </tr>
</thead>
<tbody>
<tr> <td align="center">       Pure Python        </td> <td align="center">1.0 </td> </tr>
<tr> <td align="center">   Cython                 </td> <td align="center">   0.0048    </td> </tr>
<tr> <td align="center">   Numpy               </td> <td align="center">   0.0046    </td> </tr>
</tbody>
</table>

## Cython summary

* Cython pros and cons
    * [+] Allows incremental optimization, easy to access C libraries, generated C code more compact and readable than swig, active developer community, advanced and flexible
    * [-] Less suitable than Swig for wrapping large libraries to Python modules, fully optimized code not as readable as Python
* Should be considered (maybe as a first choice?) for mixing Python with C